In [1]:
from FRTBUtils import FRTBInit
from IPython.display import display, Math
import pandas as pd
import matplotlib.pyplot as plt

#https://www.bis.org/bcbs/publ/d352.pdf#page=24&zoom=100,72,705


In [7]:
#SAMR_SBM:  Standardised Approach for Market Risk_ Sensitivity Base Method
class SAMR_SBM(FRTBInit):
    def calcRiskClassCapital(self):
        tasks = self.prepareData()
        rhoovaresult = self.getResults(tasks)
        ir_riskfactor = self.getIRRiskFactor(rhoovaresult)
        currencies= self.getCurrencies(ir_riskfactor)
        buckets=self.getBuckets(rhoovaresult,ir_riskfactor,currencies)
        sensitivities=self.calculateSensitivities(buckets,ir_riskfactor)
        weighted_sensitivities=self.applyRiskWeights(sensitivities)
        
        delta=self.calculateDelta(weighted_sensitivities)
        riskcapital=self.RiskCapital(delta)
        result={}
        result["tasks"]=tasks
        result["buckets"]=buckets
        result["sensitivities"]=sensitivities
        result["weighted_sensitivities"]=weighted_sensitivities
        result["delta"]=delta
        result["riskcapital"]=riskcapital
        return result

In [8]:
reportingdate="2021-01-28"
reportingcurrency="TRY"
results=SAMR_SBM(reportingdate,reportingcurrency).calcRiskClassCapital()

In [12]:
#https://github.com/rhoova/rhoova-examples


#We use Rhoova for sensitivity calculation based on tenor. Rhoova generates cashflows for different products 
#based on a given list of tenors (change self.timebuckets in FRTBInit.py). You can see cashflows by maturity, 
#yield curve and currency from Rhoova by calling results.get("buckets"). 

#See detailed examples of cashflow mapping.
#https://github.com/rhoova/rhoova-examples/tree/main/python_examples/Portfolio/Value%20at%20Risk/Delta%20Normal

# We use the results from rhoova in the calculateSensitivities nunction (FRTBInit.py) and
#calculate the sensitivities for each tenor.


formula = r"s_{k,r_t} = \frac{V_i(r_t + 0.0001, cs_t) - V_i(r_t, cs_t)}{0.0001}"
display(Math(formula))


sensitivities=results.get("sensitivities")
sensitivities_df = pd.concat(sensitivities, axis=1)
sensitivities_df.columns = [f"{col} Sensitivity" for col in sensitivities.keys()] 
sensitivities_df.T


<IPython.core.display.Math object>

tenors,0D,1D,3M,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,25Y,30Y,40Y,50Y
TRY Sensitivity,0.0,0.0000,13818.070981,35057.079896,108897.393328,317249.684255,528443.337378,802771.980815,5.238864e+06,6.084664e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USD Sensitivity,0.0,47802.1698,136338.737252,-16540.915987,-59952.915410,-174660.227418,-290931.837177,-441962.100178,-2.884230e+06,-3.349881e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#After calculating the sensitivity, we calculate the weighted sensitivity with the data coming from the regulator.
#Data from the regulator are hypothetical.

formula = r"WS_k = RW_k s_k"
display(Math(formula))

sensitivities=results.get("weighted_sensitivities")
sensitivities_df = pd.concat(sensitivities, axis=1)
sensitivities_df.columns = [f"{col} Weigted Sensitivity" for col in sensitivities.keys()] 
sensitivities_df.T


<IPython.core.display.Math object>

,0D,1D,3M,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,25Y,30Y,40Y,50Y
TRY Weigted Sensitivity,0.0,0.000000,1.340353,2.769509,9.909663,32.676717,78.209614,199.087451,1645.003352,2585.982248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USD Weigted Sensitivity,0.0,3.107141,13.224858,-1.306732,-5.455715,-17.990003,-43.057912,-109.606601,-905.648371,-1423.699597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Calculate deltas using correlation data

formula = r"K_b = \sqrt{\sum_{k=1}^{n} WS_k^2 + \sum_{k=1}^{n-1} \sum_{l=k+1}^{n} \rho_{kl} WS_k WS_l}"
display(Math(formula))

results.get("delta")

<IPython.core.display.Math object>

,bucket,level,Kb,Sb
0,TRY,Low,4388.779391,4554.978908
1,TRY,Medium,4466.314680,4554.978908
2,TRY,High,4542.526732,4554.978908
3,USD,Low,2414.413390,-2490.432933
4,USD,Medium,2456.526803,-2490.432933
5,USD,High,2497.930312,-2490.432933


In [21]:
formula = r"""
\text{RiskCapital} = \sqrt{\sum_{i=1}^{m} K_i^2 + \sum_{i=1}^{m-1} \sum_{j=i+1}^{m} \gamma_{ij} S_i S_j}
"""

display(Math(formula))


#Calculate Risk capital
results.get("riskcapital")

<IPython.core.display.Math object>

{'Medium': 6448.930943822323,
 'Low': 6597.951461646266,
 'High': 6296.38445031258}